In [3]:
import json
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
import spacy
import jovian
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error

In [4]:
#opening already translated and tagged dataset
with open('translated_dataframe.json', 'r', encoding='utf-8') as file:
    translated_data = json.load(file)

In [5]:
translated_df = pd.DataFrame(translated_data)

In [6]:
#need this bc too lasy to rewrite the json
translated_df['tag'][130] = str(2)
translated_df['tag'][157] = str(2)
translated_df['tag'][530] = str(4)
translated_df['tag'][653] = str(1)
translated_df['tag'][813] = str(1)

In [ ]:
#tokenization
tok = spacy.load('en')
def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in tok.tokenizer(nopunct)]